In [1]:
#插入第三方库
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import datetime
#数据可视化库
from matplotlib import pylab as plt
import seaborn as sns
import datetime
import time
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
import xgboost as xgb

from xgboost.sklearn import XGBClassifier
import operator

/Users/88michael/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#加载数据集
store=pd.read_csv("store.csv", low_memory=False)
#read_csv参数：parse_dates解析索引,index_col用作行索引的列编号，low_memory分块加载到内存，再低内存消耗中解析
train=pd.read_csv("train.csv", parse_dates=[2], low_memory=False)     #,index_col='Date')
test=pd.read_csv('test.csv', parse_dates=[3],low_memory=False)

In [3]:
store['CompetitionDistance'].fillna(store['CompetitionDistance'].median(), inplace=True)

In [4]:
store.fillna(0,inplace=True)

In [5]:
test.fillna(1,inplace=True)

In [6]:
#合并数据集store&train为新的数据训练集train_new
train=pd.merge(train, store, on='Store')
test = pd.merge(test, store, on='Store')

In [7]:
#提取开门并有销售额特征，填充缺失值
train=train[train["Open"]!=0]
train = train[train["Sales"]>0]

In [8]:
def features_create(data):
    
    
    mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    data.StoreType.replace(mappings, inplace=True)
    data.Assortment.replace(mappings, inplace=True)
    data.StateHoliday.replace(mappings, inplace=True)
    
    
    data['Year'] = data.Date.dt.year
    data['Month'] = data.Date.dt.month
    data['Day'] = data.Date.dt.day
    data['DayOfWeek'] = data.Date.dt.dayofweek
    data['WeekOfYear'] = data.Date.dt.weekofyear
    
    
    
    data['CompetitionOpen'] = 12 * (data.Year - data.CompetitionOpenSinceYear) +         (data.Month - data.CompetitionOpenSinceMonth)
    data['PromoOpen'] = 12 * (data.Year - data.Promo2SinceYear) +         (data.WeekOfYear - data.Promo2SinceWeek) / 4.0
    data['CompetitionOpen'] = data.CompetitionOpen.apply(lambda x: x if x > 0 else 0)        
    data['PromoOpen'] = data.PromoOpen.apply(lambda x: x if x > 0 else 0)
    
    
  
    month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
    data['monthStr'] = data.Month.map(month2str)
    data.loc[data.PromoInterval == 0, 'PromoInterval'] = ''
    data['IsPromoMonth'] = 0
    for interval in data.PromoInterval.unique():
        if interval != '':
            for month in interval.split(','):
                data.loc[(data.monthStr == month) & (data.PromoInterval == interval), 'IsPromoMonth'] = 1

    return data

In [9]:
features_create(train)
features_create(test)
print('Features creation finished')

Features creation finished


In [10]:
train = train.sort_values(['Date'],ascending = False)
valid = train[:6*7*1115]
train = train[6*7*1115:]

In [11]:
train.drop(['PromoInterval'],axis=1,inplace=True)
#test.drop(['PromoInterval'],axis=1,inplace=True)

In [12]:
train.drop(['Date'],axis=1,inplace=True)
#test.drop(['Date'],axis=1,inplace=True)

In [13]:
train.drop(['Customers'],axis=1,inplace=True)
#test.drop(['Id'],axis=1,inplace=True)

In [14]:
train.drop(['Open'],axis=1,inplace=True)
#test.drop(['Open'],axis=1,inplace=True)

In [15]:
train.drop(['monthStr'],axis=1,inplace=True)
#test.drop(['monthStr'],axis=1,inplace=True)

In [16]:
from sklearn.preprocessing import MinMaxScaler
from random import shuffle
from xgboost.sklearn import XGBClassifier
from sklearn.cross_validation import cross_val_score
import pickle
import time
from hyperopt import fmin,tpe,hp,space_eval,rand,Trials,partial,STATUS_OK

In [17]:
label=train.values[:,-1]
attrs=train.values[:,:-1]
labels=label.reshape((1,-1))
label=labels.tolist()[0]
minmaxscaler=MinMaxScaler()
attrs=minmaxscaler.fit_transform(attrs)



In [18]:
import random
index=range(0,len(label))
#random.shuffle(index)
trainIndex=index[:int(len(label)*0.7)]
print(len(trainIndex))
validIndex=index[int(len(label)*0.7):]
print(len(validIndex))
attr_train=attrs[trainIndex,:]
print(attr_train.shape)
attr_valid=attrs[validIndex,:]
print(attr_valid.shape)
label_train=labels[:,trainIndex].tolist()[0]
print(len(label_train))
label_valid=labels[:,validIndex].tolist()[0]
print(len(label_valid))
print(np.mat(label_train).reshape(-1,1).shape)

558255
239253
(558255, 20)
(239253, 20)
558255
239253
(558255, 1)


In [19]:
def GBM(argsDict):
    max_depth = argsDict["max_depth"] + 5
    n_estimators = argsDict['n_estimators'] * 5 + 50
    learning_rate = argsDict["learning_rate"] * 0.02 + 0.05
    subsample = argsDict["subsample"] * 0.1 + 0.7
    min_child_weight = argsDict["min_child_weight"]+1
    print("max_depth:" + str(max_depth))
    print("n_estimator:" + str(n_estimators))
    print("learning_rate:" + str(learning_rate))
    print("subsample:" + str(subsample))
    print("min_child_weight:" + str(min_child_weight))
    global attr_train,label_train

    gbm = xgb.XGBClassifier(nthread=4,    #进程数
                            max_depth=max_depth,  #最大深度
                            n_estimators=n_estimators,   #树的数量
                            learning_rate=learning_rate, #学习率
                            subsample=subsample,      #采样数
                            min_child_weight=min_child_weight,   #孩子数
                            max_delta_step = 10,  #10步不降则停止
                            objective="reg:linear")

    metric = cross_val_score(gbm,attr_train,label_train,cv=5,scoring="r2").mean()
    print(metric)
    return -metric

space = {"max_depth":hp.randint("max_depth",15),
         "n_estimators":hp.randint("n_estimators",10),  #[0,1,2,3,4,5] -> [50,]
         "learning_rate":hp.randint("learning_rate",6),  #[0,1,2,3,4,5] -> 0.05,0.06
         "subsample":hp.randint("subsample",4),#[0,1,2,3] -> [0.7,0.8,0.9,1.0]
         "min_child_weight":hp.randint("min_child_weight",5), #
        }
algo = partial(tpe.suggest,n_startup_jobs=1)
best = fmin(GBM,space,algo=algo,max_evals=4)

print(best)
print(GBM(best))

max_depth:14
n_estimator:95
learning_rate:0.05
subsample:0.7999999999999999
min_child_weight:1


/Users/88michael/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/88michael/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/88michael/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/88michael/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an 

0.17048786263262752
max_depth:14
n_estimator:95
learning_rate:0.05
subsample:0.7999999999999999
min_child_weight:1


/Users/88michael/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/88michael/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/88michael/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/88michael/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an 

0.17048786263262752
max_depth:13
n_estimator:50
learning_rate:0.09
subsample:0.7999999999999999
min_child_weight:3


...

/Users/88michael/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/88michael/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/88michael/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/88michael/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an 

0.15359949224290387
{'max_depth': 8, 'subsample': 1, 'min_child_weight': 2, 'n_estimators': 0, 'learning_rate': 2}
max_depth:13
n_estimator:50
learning_rate:0.09
subsample:0.7999999999999999
min_child_weight:3


/Users/88michael/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/88michael/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/88michael/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/88michael/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an 

0.18347888920656855
-0.18347888920656855


/Users/88michael/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
